In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_current
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH ctl AS (
    SELECT DISTINCT 
        store_id, product_id
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd
    WHERE date(start_date) <= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
      AND date(end_date) >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
),
ain AS (
    SELECT DISTINCT 
        store_id, product_id
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd
    WHERE date(start_date) <= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
      AND date(end_date) >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
      AND status = 'ACTIVE'
),
aou AS (
    SELECT DISTINCT 
        store_id, product_id
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd
    WHERE date(start_date) <= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
      AND date(end_date) >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
      AND status = 'INACTIVE'
),
clst AS (
    SELECT DISTINCT 
        store_id, product_id
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd
    WHERE date(start_date) <= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
      AND date(end_date) >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
      AND status = 'CLEARSTOCK'
),
ativ AS (
    SELECT 
        ain.store_id, 
        ain.product_id
    FROM ain
    LEFT JOIN aou 
      ON ain.store_id = aou.store_id AND ain.product_id = aou.product_id
    WHERE aou.store_id IS NULL
)
SELECT 
    st.business_unit,
    IFNULL(st.region_domain_vn, '') AS mien, 
    IFNULL(st.city, '') AS city,
    IFNULL(st.district, '') AS district,
    ctl.store_id, 
    st.store_name,
    ctl.product_id,
    pr.product_name,
    pr.manufacturer_id,
    pr.manufacturer_name,
    pr.mch2_id,
    pr.mch3_id,
    IF(ativ.product_id IS NOT NULL, 'ACTIVE', 'INACTIVE') AS status
FROM ctl
LEFT JOIN ativ 
  ON ctl.store_id = ativ.store_id AND ctl.product_id = ativ.product_id
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st 
  ON ctl.store_id = st.store_id
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product pr 
  ON ctl.product_id = pr.product_id
""")
